In [8]:


import pandas as pd
import numpy as np

# 데이터 준비
df = pd.read_csv("data/credit_card_churn.csv")
df.head()

# Delete the 'Gender' column
df.drop(columns=['CLIENTNUM', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], inplace=True)
df.head()

def find_outliers(df, column_name, whis=1.5):
    """
    분위수 기준으로 이상치를 찾는 함수

    Parameters:
    df (pd.DataFrame): 데이터프레임
    column_name (str): 이상치를 찾을 컬럼명

    Returns:
    pd.Series: 이상치 값들
    """
    q1, q3 = df[column_name].quantile(q=[0.25, 0.75])
    iqr = q3 - q1
    iqr *= whis
    return df.loc[~df[column_name].between(q1 - iqr, q3 + iqr), column_name].index

# 언노운 값을 NA 값으로 변환 후 최빈값으로 결측치 처리,  income unknown 은 레이블 인코딩 후 결측치 처리
df2 = df.copy()
df2["Education_Level"]=df2["Education_Level"].replace("Unknown",pd.NA)
df2["Marital_Status"]=df2["Marital_Status"].replace("Unknown",pd.NA)
# df2["Income_Category"]=df2["Income_Category"].replace("Unknown",pd.NA)
df2["Education_Level"]=df2["Education_Level"].fillna(df['Education_Level'].mode()[0])
df2["Marital_Status"]=df2["Marital_Status"].fillna(df['Marital_Status'].mode()[0])


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

category_columns = ['Gender','Education_Level','Marital_Status','Income_Category', 'Card_Category']
number_columns = ['Customer_Age', 'Months_on_book','Credit_Limit','Total_Revolving_Bal','Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio' ]
# 'Total_Relationship_Count','Months_Inactive_12_mon','Contacts_Count_12_mon','Dependent_count' 제외
df3 = df2.copy() 
le_dict = {}
for col in category_columns:
    le = LabelEncoder()
    df3[col] = le.fit_transform(df3[col])  # df3 : 결측치 처리완료한 데이터
    le_dict[col] = le # dictionary[컬럼명, LabelEncoder객체]

for col2 in number_columns:
    # 객체 생성
    s_scaler = StandardScaler()
    df3[[col2]] = s_scaler.fit_transform(df3[[col2]]) # 학습/변환 대상이 같은 경우.

# df3["Attrition_Flag"]=df3["Education_Level"].replace("Existing Customer",0)
# df3["Attrition_Flag"]=df3["Education_Level"].replace("Attrited Customer",1)
df3



,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,3,-0.165406,1,3,3,1,2,0,0.384621,5,1,3,0.446622,-0.473422,0.488971,2.623494,-0.959707,-0.973895,3.834003,-0.775882
1,2,0.333570,0,5,2,2,4,0,1.010715,6,1,2,-0.041367,-0.366667,-0.008486,3.563293,-0.916433,-1.357340,12.608573,-0.616276
2,2,0.583058,1,3,2,1,3,0,0.008965,4,1,0,-0.573698,-1.426858,-0.445658,8.367214,-0.740982,-1.911206,6.807864,-0.997155
3,3,-0.789126,0,4,3,1,4,0,-0.241473,3,4,1,-0.585251,1.661686,-0.734100,2.942843,-0.951758,-1.911206,6.807864,1.759686
4,5,-0.789126,1,3,5,1,2,0,-1.869317,5,1,0,-0.430877,-1.426858,-0.302868,6.455682,-1.056263,-1.570365,7.509325,-0.997155
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10122,2,0.458314,1,2,2,2,1,0,0.509840,3,2,3,-0.509330,0.844455,-0.584929,-0.259771,3.259358,2.221481,0.608119,0.678714
10123,2,-0.664382,1,2,2,0,1,0,-1.368442,4,2,3,-0.479181,1.255524,-0.591639,0.201004,1.283475,0.176440,-0.122745,0.856458
10124,3,-0.290150,0,1,3,1,4,0,0.008965,5,3,4,-0.354626,-1.426858,-0.226632,0.269436,1.732994,-0.207005,0.444305,-0.997155
10125,2,-2.036565,1,2,2,1,1,0,0.008965,4,3,3,-0.368710,-1.426858,-0.240713,-1.026208,1.174848,-0.121795,0.041070,-0.997155
